# Mahnob Baseline

Using leave one subject out, evaluate EEG-only spectral power and vgg topo features.

In [1]:
import sys
sys.path.insert(0, "../")

In [2]:
%pip install git+https://github.com/masaponto/python-elm

  Cloning https://github.com/masaponto/python-elm to /tmp/pip-req-build-z7jky97w
  Running command git clone --filter=blob:none --quiet https://github.com/masaponto/python-elm /tmp/pip-req-build-z7jky97w
  Resolved https://github.com/masaponto/python-elm to commit b253b5c262efeb1f8eeb5e14b55f98778409b54d
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

from libs.dataloaders import mahnob
import xml.etree.ElementTree as ET

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler

from elm import ELM
from sklearn.preprocessing import normalize

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-753n1gzq because the default path (/home/anp054/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
def get_sessions_subjs():
    path = "/net2/expData/affective_eeg/mahnob_dataset/Sessions"
    sessions = os.listdir(path)
    subjs = {}
    scores = {}
    for session in sessions:
        mytree = ET.parse(f'{path}/{session}/session.xml')
        myroot = mytree.getroot()
        if myroot[0].attrib['id'] not in subjs:
            subjs[myroot[0].attrib['id']] = []
        if 'feltVlnc' in myroot.attrib:
            subjs[myroot[0].attrib['id']].append(session)
            scores[session] = int(myroot.attrib['feltVlnc'])
    return subjs, scores
subjects, subject_scores = get_sessions_subjs()

In [5]:
def subject_cv(dataset_vgg, subjects):
    """
    Run per subject cross-validation using ELM (sweeping neuron size 1-200) and LDA.
    Notes:
        - Do _not_ enforce balancing on training, but set equal priors for LDA (data is roughly balanced)
        - Balance test data; skip when per-label count < 3
        - When sweeping ELM neurons, fix the error in the original paper implementation where they
          are fitting to test accuracy. Instead tune on training, and then evaluate on test.
    """
    lda_acc, elm_acc = [], []
    for sessions in tqdm(subjects.values()):
        train_data = [dataset_vgg[i] for i in range(len(dataset_vgg)) if dataset_vgg.sessions[i] not in sessions]
        test_data = [dataset_vgg[i] for i in range(len(dataset_vgg)) if dataset_vgg.sessions[i] in sessions]
        
        X_train_vgg = np.array([i[0].flatten() for i in train_data])
        Y_train_vgg = np.squeeze([i[1] for i in train_data])*2-1 # rescale to -1, 1
        X_test_vgg = np.array([i[0].flatten() for i in test_data])
        Y_test_vgg = np.squeeze([i[1] for i in test_data])*2-1 # rescale to -1, 1

        if len(Y_train_vgg) == 0 or len(Y_test_vgg) == 0:
            continue
        if len(np.unique(Y_test_vgg, return_counts=True)[-1]) != len(np.unique(Y_train_vgg, return_counts=True)[-1]):
            print("Skipping due to num classes not being equivalent in train and test")
            continue
        
        # This really only works for 2 classes; balance
        u_val, u_count = np.unique(Y_test_vgg, return_counts=True)
        u_count = list(u_count)
        if u_count[0] != u_count[1]:
            u_reduce = u_val[u_count.index(max(u_count))]
            u_reduce_count = abs(u_count[0] - u_count[1])
            match_idxs = np.argwhere(Y_test_vgg == u_reduce).squeeze()
            np.random.shuffle(match_idxs)
            remove_idxs = match_idxs[:u_reduce_count]
            keep_idxs = [i for i in range(len(Y_test_vgg)) if i not in remove_idxs]
            X_test_vgg = X_test_vgg[keep_idxs]
            Y_test_vgg = Y_test_vgg[keep_idxs]
        
        if min(np.unique(Y_test_vgg, return_counts=True)[-1]) < 3:
            print("Skipping due to fewer than 3 samples per class")
            continue

        # Shuffle train
        shuffle = np.arange(X_train_vgg.shape[0])
        np.random.shuffle(shuffle)
        X_train_vgg = X_train_vgg[shuffle]
        Y_train_vgg = Y_train_vgg[shuffle]

        pca = PCA(n_components=30)
        pca.fit(X_train_vgg)
        X_train_vgg = pca.transform(X_train_vgg)
        X_test_vgg = pca.transform(X_test_vgg)

        lda = LinearDiscriminantAnalysis(priors=(0.5,0.5))
        lda.fit(X_train_vgg, Y_train_vgg)
        lda_test = lda.score(X_test_vgg, Y_test_vgg)

        best_q = (0, 0) # neuron, train_perf
        for q in range(1,200):
            elm = ELM(hid_num=q).fit(X_train_vgg, Y_train_vgg)
            score = elm.score(X_train_vgg, Y_train_vgg)
            if score > best_q[1]:
                best_q = (q, score)
        elm = ELM(hid_num=best_q[0]).fit(X_train_vgg, Y_train_vgg)
        elm_test = elm.score(X_test_vgg, Y_test_vgg)
            
        lda_acc.append(lda_test)
        elm_acc.append(elm_test)
        # print(lda_test, elm_test)

    print(f"LDA: {np.mean(lda_acc):.2f} +/- {np.std(lda_acc)/np.sqrt(len(lda_acc)):.3f}")
    print(f"ELM: {np.mean(elm_acc):.2f} +/- {np.std(elm_acc)/np.sqrt(len(elm_acc)):.3f}")

In [6]:
import torch
def vgg16_augment(model):
    model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-3])

## Average Spectral Power

Average Theta, Alpha, Beta, Gamma features per channel. The performance results are reported as LDA and ELM (note: this ELM version uses the max neuron count performance value _per subject_ that Sid's paper used).

In [6]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "AvgFreqPower", 
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [00:34<00:00, 15.43it/s]

INFO: T dimension is not equivalent across all S; reducing to T=1


In [7]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:10<01:04,  2.57s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:15<00:36,  1.75s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:43<00:17,  2.50s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [00:59<00:00,  2.04s/it]

LDA: 0.59 +/- 0.024
ELM: 0.54 +/- 0.037


## VGG Topo Features

Average Theta, Alpha, Beta topo fed through pretrained VGG16 model. The performance results are reported as LDA and ELM (note: this ELM version uses the max neuron count performance value _per subject_ that Sid's paper used).

In [7]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "vgg16",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment,
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [02:25<00:00,  3.62it/s]

INFO: T dimension is not equivalent across all S; reducing to T=1


In [8]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:10<01:06,  2.66s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:16<00:38,  1.83s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:45<00:18,  2.61s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [01:01<00:00,  2.12s/it]

LDA: 0.52 +/- 0.018
ELM: 0.51 +/- 0.033


# Window Evaluation

## Average Spectral Power

In [10]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "AvgFreqPower", 
        "window": 2560, # 10s window
        "stride": 2560, # 10s stride
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [00:35<00:00, 14.72it/s]


INFO: T dimension is not equivalent across all S; reducing to T=9


In [11]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:10<01:06,  2.65s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:15<00:38,  1.81s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:44<00:17,  2.54s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [01:00<00:00,  2.08s/it]

LDA: 0.61 +/- 0.026
ELM: 0.54 +/- 0.034


## VGG Topo Features

In [12]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "vgg16",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment,
        "window": 2560, # 10s window
        "stride": 2560, # 10s stride
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [25:07<00:00,  2.87s/it]

INFO: T dimension is not equivalent across all S; reducing to T=9


In [13]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:12<01:17,  3.10s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:18<00:44,  2.14s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:53<00:21,  3.02s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [01:12<00:00,  2.49s/it]

LDA: 0.57 +/- 0.020
ELM: 0.55 +/- 0.031


# Random Models

## Default

In [14]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "vgg16",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment,
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [02:19<00:00,  3.78it/s]

INFO: T dimension is not equivalent across all S; reducing to T=1


In [15]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:10<01:06,  2.65s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:16<00:38,  1.83s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:45<00:18,  2.57s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [01:01<00:00,  2.11s/it]

LDA: 0.53 +/- 0.017
ELM: 0.46 +/- 0.039


In [16]:
default_params = []
for param in dataset.x_transformer.model.parameters():
    default_params.append(param.view(-1))

## Initialization

In [17]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "vgg16",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment,
        "random_weights": {
            "mode": "rand_init"
        },
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [02:17<00:00,  3.83it/s]

INFO: T dimension is not equivalent across all S; reducing to T=1


In [18]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:10<01:04,  2.56s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:15<00:36,  1.73s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:43<00:17,  2.47s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [00:59<00:00,  2.05s/it]

LDA: 0.54 +/- 0.024
ELM: 0.54 +/- 0.042


In [19]:
init_params = []
for param in dataset.x_transformer.model.parameters():
    init_params.append(param.view(-1))

## Perturb Pretrained

In [20]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "vgg16",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment,
        "random_weights": {
            "mode": "perturb",
            "distribution": "uniform",
        },
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [02:18<00:00,  3.81it/s]

INFO: T dimension is not equivalent across all S; reducing to T=1


In [21]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:09<01:02,  2.48s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:14<00:35,  1.69s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:42<00:16,  2.39s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [00:57<00:00,  1.97s/it]

LDA: 0.48 +/- 0.025
ELM: 0.49 +/- 0.033


In [22]:
perturb_params = []
for param in dataset.x_transformer.model.parameters():
    perturb_params.append(param.view(-1))

## Shuffle Pretrained

In [23]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "vgg16",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment,
        "random_weights": {
            "mode": "shuffle"
        },
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [03:20<00:00,  2.62it/s]  

INFO: T dimension is not equivalent across all S; reducing to T=1


In [24]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:12<01:16,  3.05s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:18<00:43,  2.08s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:51<00:20,  2.91s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [01:09<00:00,  2.40s/it]

LDA: 0.52 +/- 0.020
ELM: 0.49 +/- 0.033


In [25]:
shuffled_params = []
for param in dataset.x_transformer.model.parameters():
    shuffled_params.append(param.view(-1))

## Compare

In [26]:
default_params[1]

tensor([ 0.4034,  0.3778,  0.4644, -0.3228,  0.3940, -0.3953,  0.3951, -0.5496,
         0.2693, -0.7602, -0.3508,  0.2334, -1.3239, -0.1694,  0.3938, -0.1026,
         0.0460, -0.6995,  0.1549,  0.5628,  0.3011,  0.3425,  0.1073,  0.4651,
         0.1295,  0.0788, -0.0492, -0.5638,  0.1465, -0.3890, -0.0715,  0.0649,
         0.2768,  0.3279,  0.5682, -1.2640, -0.8368, -0.9485,  0.1358,  0.2727,
         0.1841, -0.5325,  0.3507, -0.0827, -1.0248, -0.6912, -0.7711,  0.2612,
         0.4033, -0.4802, -0.3066,  0.5807, -1.3325,  0.4844, -0.8160,  0.2386,
         0.2300,  0.4979,  0.5553,  0.5230, -0.2182,  0.0117, -0.5516,  0.2108],
       grad_fn=<ViewBackward0>)

In [27]:
init_params[1]

tensor([ 0.0168,  0.1491, -0.1472,  0.0163, -0.0874,  0.0912, -0.1743,  0.0006,
         0.0752,  0.1656, -0.1164, -0.1686, -0.0331,  0.1895,  0.0928, -0.0934,
         0.1055,  0.0024,  0.1031, -0.1847,  0.1808, -0.1919, -0.0429,  0.0382,
         0.1669,  0.1425,  0.0551,  0.0876,  0.0966, -0.0807,  0.1244, -0.1771,
         0.0930,  0.1858, -0.0774,  0.1246, -0.0735, -0.0906,  0.0064, -0.0687,
         0.0907, -0.1748,  0.0366, -0.0847,  0.0744, -0.1750,  0.1017, -0.1890,
         0.1362, -0.0869, -0.0358,  0.0219,  0.0804, -0.0770, -0.0837,  0.1533,
         0.1290, -0.0759,  0.0127, -0.0010, -0.0464, -0.1196, -0.1643,  0.0142],
       grad_fn=<ViewBackward0>)

In [28]:
perturb_params[1]

tensor([ 0.4161,  0.3915,  0.4802, -0.3114,  0.4050, -0.3841,  0.4140, -0.5352,
         0.2805, -0.7452, -0.3358,  0.2523, -1.3126, -0.1548,  0.4053, -0.0873,
         0.0604, -0.6823,  0.1692,  0.5766,  0.3110,  0.3587,  0.1178,  0.4829,
         0.1464,  0.0955, -0.0379, -0.5503,  0.1631, -0.3703, -0.0585,  0.0817,
         0.2877,  0.3382,  0.5810, -1.2497, -0.8190, -0.9364,  0.1474,  0.2857,
         0.1946, -0.5137,  0.3650, -0.0650, -1.0104, -0.6750, -0.7520,  0.2712,
         0.4177, -0.4675, -0.2898,  0.5982, -1.3196,  0.4987, -0.8042,  0.2532,
         0.2447,  0.5163,  0.5682,  0.5365, -0.2059,  0.0226, -0.5352,  0.2283],
       grad_fn=<ViewBackward0>)

In [29]:
shuffled_params[1]

tensor([ 0.3425,  0.3778, -0.5496,  0.0788,  0.2727,  0.2693,  0.2386,  0.5628,
         0.1295,  0.1841, -1.2640,  0.5230,  0.5682,  0.3011,  0.0649, -1.3239,
         0.3938,  0.1465,  0.2768, -0.3508,  0.4033, -1.3325,  0.2300,  0.0117,
        -0.9485, -0.1026,  0.2334,  0.5553,  0.4844,  0.0460, -0.8160, -0.0492,
        -0.2182, -0.0827,  0.1358, -0.5325,  0.4644, -0.5638, -0.6995,  0.4651,
        -0.6912, -0.4802,  0.4034, -0.8368,  0.3507, -0.3066,  0.3951, -0.3953,
        -0.3228, -1.0248,  0.1549,  0.5807, -0.5516, -0.1694, -0.7602, -0.7711,
         0.3279, -0.0715,  0.3940,  0.2612,  0.1073, -0.3890,  0.4979,  0.2108],
       grad_fn=<ViewBackward0>)

# Alt Models

In [30]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "alexnet",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment, # using same transform for alexnet
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [01:27<00:00,  6.04it/s]

INFO: T dimension is not equivalent across all S; reducing to T=1


In [31]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:11<01:10,  2.81s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:17<00:40,  1.93s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:48<00:19,  2.75s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [01:05<00:00,  2.25s/it]

LDA: 0.55 +/- 0.025
ELM: 0.56 +/- 0.036


In [32]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapNN",
        "model": "efficientnet_v2_l",
        "model_params": {
            "weights": "DEFAULT",
        },
        "model_augment_fn": vgg16_augment, # using same transform for efficientnet
        "window": -1,
        "stride": -1
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49
)

100%|██████████| 526/526 [03:11<00:00,  2.74it/s]

INFO: T dimension is not equivalent across all S; reducing to T=1


In [33]:
subject_cv(dataset, subjects)

 14%|█▍        | 4/29 [00:11<01:08,  2.75s/it]

Skipping due to fewer than 3 samples per class


 28%|██▊       | 8/29 [00:16<00:39,  1.89s/it]

Skipping due to num classes not being equivalent in train and test
Skipping due to fewer than 3 samples per class


 76%|███████▌  | 22/29 [00:47<00:18,  2.69s/it]

Skipping due to fewer than 3 samples per class


100%|██████████| 29/29 [01:03<00:00,  2.20s/it]

LDA: 0.52 +/- 0.029
ELM: 0.58 +/- 0.024
